In [ ]:
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from itertools import groupby
from operator import itemgetter
import nltk
from nltk.stem.porter import PorterStemmer
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
p_stemmer = PorterStemmer()
nltk.download('stopwords')
stop_words = set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [108]:
class summariser:
  def __init__(self,text):

    self.text_orig = np.array(text.split("."))
    self.text_l = (re.sub(r'[^\w\s\.]', '', text.lower()).split("."))
    indexes = range(0,len(self.text_l))
    self.text_nl = np.array([np.array([ p_stemmer.stem(j) for j in self.text_l[i].split(" ") if j and j not in stop_words ],dtype=object) for i in range(len(self.text_l))],dtype=object)
    self.text_l = np.array([self.text_nl[i] for i in range(len(self.text_nl)) if  any(self.text_nl[i])],dtype=object)
    self.indexes = np.array([i for i in range(len(self.text_nl)) if  any(self.text_nl[i])])

  def build(self,tol=0.001,max_iter=50,d=0.85,n_sent=5):
    N = self.text_l.shape[0]
    sim_matrix = np.zeros((N,N))
    for i in range(N):
      for j in range(i,N):
        num = len(set(self.text_l[i]).intersection(set(self.text_l[j])))
        sim_matrix[i][j] = sim_matrix[j][i] = num/(np.log(len(self.text_l[i])+1)+np.log(len(self.text_l[j])+1)+1)
    
    scores = np.ones(N)
    count = 0
    while count < max_iter:
      old_scores = scores[:]
      for i in range(N):
        scores[i] = (1-d) +   d*((sim_matrix[i]*scores)/(sim_matrix.sum(axis=1))).sum()
      
      if np.linalg.norm(old_scores-scores) < tol :
        break
      count += 1
    sorted_indx = np.argsort(scores)
    output_indx = self.indexes[sorted_indx[-n_sent:]]
    summary_sent = self.text_orig[sorted(output_indx)]
    self.scores = scores
    self.sim_matrix = sim_matrix
    self.output_indx = output_indx
    self.sorted_indx = sorted_indx
    return summary_sent


In [100]:
data = """Junk foods taste good that’s why it is mostly liked by everyone of any age group especially kids and school going children. They generally ask for the junk food daily because they have been trend so by their parents from the childhood. They never have been discussed by their parents about the harmful effects of junk foods over health. According to the research by scientists, it has been found that junk foods have negative effects on the health in many ways. They are generally fried food found in the market in the packets. They become high in calories, high in cholesterol, low in healthy nutrients, high in sodium mineral, high in sugar, starch, unhealthy fat, lack of protein and lack of dietary fibers. Processed and junk foods are the means of rapid and unhealthy weight gain and negatively impact the whole body throughout the life. It makes able a person to gain excessive weight which is called as obesity. Junk foods tastes good and looks good however do not fulfil the healthy calorie requirement of the body. Some of the foods like french fries, fried foods, pizza, burgers, candy, soft drinks, baked goods, ice cream, cookies, etc are the example of high-sugar and high-fat containing foods. It is found according to the Centres for Disease Control and Prevention that Kids and children eating junk food are more prone to the type-2 diabetes. In type-2 diabetes our body become unable to regulate blood sugar level. Risk of getting this disease is increasing as one become more obese or overweight. It increases the risk of kidney failure. Eating junk food daily lead us to the nutritional deficiencies in the body because it is lack of essential nutrients, vitamins, iron, minerals and dietary fibers. It increases risk of cardiovascular diseases because it is rich in saturated fat, sodium and bad cholesterol. High sodium and bad cholesterol diet increases blood pressure and overloads the heart functioning. One who like junk food develop more risk to put on extra weight and become fatter and unhealthier. Junk foods contain high level carbohydrate which spike blood sugar level and make person more lethargic, sleepy and less active and alert. Reflexes and senses of the people eating this food become dull day by day thus they live more sedentary life. Junk foods are the source of constipation and other disease like diabetes, heart ailments, clogged arteries, heart attack, strokes, etc because of being poor in nutrition. Junk food is the easiest way to gain unhealthy weight. The amount of fats and sugar in the food makes you gain weight rapidly. However, this is not a healthy weight. It is more of fats and cholesterol which will have a harmful impact on your health. Junk food is also one of the main reasons for the increase in obesity nowadays.This food only looks and tastes good, other than that, it has no positive points. The amount of calorie your body requires to stay fit is not fulfilled by this food. For instance, foods like French fries, burgers, candy, and cookies, all have high amounts of sugar and fats. Therefore, this can result in long-term illnesses like diabetes and high blood pressure. This may also result in kidney failure. Above all, you can get various nutritional deficiencies when you don’t consume the essential nutrients, vitamins, minerals and more. You become prone to cardiovascular diseases due to the consumption of bad cholesterol and fat plus sodium. In other words, all this interferes with the functioning of your heart. Furthermore, junk food contains a higher level of carbohydrates. It will instantly spike your blood sugar levels. This will result in lethargy, inactiveness, and sleepiness. A person reflex becomes dull overtime and they lead an inactive life. To make things worse, junk food also clogs your arteries and increases the risk of a heart attack. Therefore, it must be avoided at the first instance to save your life from becoming ruined.The main problem with junk food is that people don’t realize its ill effects now. When the time comes, it is too late. Most importantly, the issue is that it does not impact you instantly. It works on your overtime; you will face the consequences sooner or later. Thus, it is better to stop now.You can avoid junk food by encouraging your children from an early age to eat green vegetables. Their taste buds must be developed as such that they find healthy food tasty. Moreover, try to mix things up. Do not serve the same green vegetable daily in the same style. Incorporate different types of healthy food in their diet following different recipes. This will help them to try foods at home rather than being attracted to junk food.In short, do not deprive them completely of it as that will not help. Children will find one way or the other to have it. Make sure you give them junk food in limited quantities and at healthy periods of time. """

In [109]:
s = summariser(data)
summ = s.build(max_iter = 100,tol=0.000001)

In [110]:
summ

array([' Processed and junk foods are the means of rapid and unhealthy weight gain and negatively impact the whole body throughout the life',
       ' Eating junk food daily lead us to the nutritional deficiencies in the body because it is lack of essential nutrients, vitamins, iron, minerals and dietary fibers',
       ' One who like junk food develop more risk to put on extra weight and become fatter and unhealthier',
       ' Junk foods contain high level carbohydrate which spike blood sugar level and make person more lethargic, sleepy and less active and alert',
       ' To make things worse, junk food also clogs your arteries and increases the risk of a heart attack'],
      dtype='<U183')

In [ ]:
summ

array([' Processed and junk foods are the means of rapid and unhealthy weight gain and negatively impact the whole body throughout the life',
       ' Eating junk food daily lead us to the nutritional deficiencies in the body because it is lack of essential nutrients, vitamins, iron, minerals and dietary fibers',
       ' One who like junk food develop more risk to put on extra weight and become fatter and unhealthier',
       ' Junk foods contain high level carbohydrate which spike blood sugar level and make person more lethargic, sleepy and less active and alert',
       ' To make things worse, junk food also clogs your arteries and increases the risk of a heart attack'],
      dtype='<U183')